In [59]:
import numpy as np

import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

from scipy.ndimage import imread, generic_filter
from scipy.misc import imresize
from scipy import stats

from sklearn.cluster import KMeans
from sklearn import linear_model

from scipy.ndimage import imread, generic_filter
from scipy.misc import imresize
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [27]:
# extract feature 1: asymmetry

def feature_asymmetry(img_path):
    
    img = imread(img_path)
    
    (x,y,c) = img.shape
    points = img.reshape(x * y, c)
    mat = np.mat(points)
    
    model = KMeans(n_clusters=2)
    model.fit(mat)
    labels = model.labels_

    # ensure that 0 is the label for bright points
    (center0, center1) = model.cluster_centers_
    def average_brightness(values):
        return np.mean(values)

    def switch_labels(labels):
        return np.array([1 - label for label in labels])

    if (average_brightness(center0) < average_brightness(center1)):
        labels = switch_labels(labels)
    
    labeled_img = labels.reshape(x, y)
    
    (height, width) = labeled_img.shape
    width_center = width//2
    labeled_img_l = labeled_img[:,0:width_center-1]
    
    # print(width)
    # print(width_center)
    
    # import pdb; pdb.set_trace() # debugging starts here
    
    if width % 2 == 0:
        labeled_img_r = labeled_img[:,width_center+1:]
    else:
        labeled_img_r = labeled_img[:,width_center+2:]    

    labeled_img_r_flip = np.fliplr(labeled_img_r)
    
    labeled_img_diff = (labeled_img_l - labeled_img_r_flip) ** 2   # compute image of squared error
    
    return np.sum(labeled_img_diff) / (height*width)

In [55]:
# extract feature 2: color inhomogenity

def feature_colin(img_path):
    
    img = imread(img_path)
    
    (x,y,c) = img.shape
    points = img.reshape(x * y, c)
    mat = np.mat(points)
    
    model = KMeans(n_clusters=2)
    model.fit(mat)
    labels = model.labels_

    # ensure that 0 is the label for bright points
    (center0, center1) = model.cluster_centers_
    def average_brightness(values):
        return np.mean(values)

    def switch_labels(labels):
        return np.array([1 - label for label in labels])

    if (average_brightness(center0) < average_brightness(center1)):
        labels = switch_labels(labels)
    
    labeled_img = labels.reshape(x, y)
    
    img_gs = imread(img_path,mode='L')
    
    img_mole = img_gs   # load grayscale image
    img_mole[labeled_img==0] = 0
    
    img_mole_vec = np.reshape(img_mole,(1,-1))   # matrix to vector
    img_mole_vec_rm = img_mole_vec[img_mole_vec > 0]  # remove zero elements (skin)

    return np.var(img_mole_vec_rm)

In [28]:
# load data

path_common = "D:\\Hackathon\\mole classification\\data\\ph2 sorted\\common"
path_atypical = "D:\\Hackathon\\mole classification\\data\\ph2 sorted\\atypical"
path_melanoma = "D:\\Hackathon\\mole classification\\data\\ph2 sorted\\melanoma"

imgs_common =   [join(path_common, f)   for f in listdir(path_common)   if isfile(join(path_common, f))]
imgs_atypical = [join(path_atypical, f) for f in listdir(path_atypical) if isfile(join(path_atypical, f))]
imgs_melanoma = [join(path_melanoma, f) for f in listdir(path_melanoma) if isfile(join(path_melanoma, f))]

imgs = imgs_common + imgs_melanoma

Y_0 = np.squeeze(np.tile(0,(len(imgs_common),1)))
Y_1 = np.squeeze(np.tile(1,(len(imgs_melanoma),1)))
Y = np.concatenate([Y_0,Y_1])

In [30]:
X = []
for i in range(0,len(imgs)):
    X.append(feature_asymmetry(imgs[i]))

In [63]:
X_1 = np.asarray(X)
X_1 = stats.zscore(X_1)

In [56]:
colins = []
for i in range(0,len(imgs)):
    colins.append(feature_colin(imgs[i]))

In [81]:
X_2 = np.asarray(colins)
X_2 = stats.zscore(X_2)

In [92]:
X_all = np.vstack([X_1, X_2])

In [95]:
# unfinished code until here

<function ndarray.transpose>

In [85]:
model = linear_model.LogisticRegression(C=1e5)

In [86]:
# we create an instance of Neighbours Classifier and fit the data.
model.fit(X_all, Y)

ValueError: Found arrays with inconsistent numbers of samples: [  2 120]